In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from books import get_book_dataframe, get_book_features
data_path = '../data/goodbooks-10k/'
book_features = get_book_features(get_book_dataframe(data_path))

In [4]:
from joiner import get_ratings, get_reduced_joint
goodreads_path = '../data/goodbooks-10k/ratings.csv'
amazon_path = '../data/amazon/ratings_amazon.csv'
spr = get_ratings(goodreads_path, amazon_path, min_amazon_items = 5)

/mnt/c/Users/doruk/Desktop/Playground/book-recommender-system/Evaluation/util.py:48: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  row = ratings.user_id.astype('category', categories=users).cat.codes
/mnt/c/Users/doruk/Desktop/Playground/book-recommender-system/Evaluation/util.py:49: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  col = ratings.book_id.astype('category', categories=books).cat.codes


In [5]:
features_components = 300
ratings_components = 300
learning_rate = 0.01
batch_size = 128

In [6]:
from util import get_sparse, reduce_matrix
reduced_item_features, _, _ = reduce_matrix(book_features, n_components = features_components)

In [8]:
from cross_validation import ColumnwiseKFold
n_folds = 5
kf = ColumnwiseKFold(n_folds, random_seed = 30)
a = [a for a in enumerate(kf.split(spr))]
a = a[0]
i, (X, (user_incides, item_indices)) = a

In [9]:
_, _, rating_VT = reduce_matrix(X, n_components = ratings_components)
reduced_item_ratings = rating_VT.T

In [10]:
items = get_reduced_joint(reduced_item_ratings, reduced_item_features)

In [11]:
tf_items = tf.placeholder(tf.float32, (None, items.shape[1]))

In [12]:
enc1 = tf.layers.dense(tf_items, 200, activation = tf.nn.relu)
enc2 = tf.layers.dense(enc1, 150, activation = tf.nn.relu)

In [13]:
dec1 = tf.layers.dense(enc1, 200, activation = tf.nn.relu)
dec2 = tf.layers.dense(enc1, items.shape[1], activation = tf.nn.relu)

In [14]:
reconstruction_loss = tf.reduce_mean(tf.pow(tf_items - dec2, 2))

In [15]:
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(reconstruction_loss)

In [16]:
from math import ceil
n_train = items.shape[0]
n_batches = ceil(n_train * 1.0 / batch_size)

In [17]:
x_items = [items[i * batch_size: min((i + 1) * batch_size, n_train)] for i in range(n_batches)]
sess = tf.Session()

In [18]:
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())
for i in range(3000):
    _, l = sess.run([optimizer, reconstruction_loss],
                    feed_dict={
                        tf_items: x_items[i % len(x_items)]})
    if i % 100 == 0:
        print(l)

0.005409082
0.0035899584
0.0023452835
0.0018511331
0.0019377748
0.0017509753
0.0018621528
0.0017680447
0.0016930173
0.0016889831
0.001675233
0.0015903157
0.0016270886
0.0016930905
0.0015557241
0.0016346042
0.001701319
0.0016300523
0.0016291173
0.0015790871
0.0015808585
0.0014857096
0.0015043818
0.0015300049
0.0016117733
0.0015859832
0.0014950776
0.0015603435
0.00151159
0.0015609588


In [19]:
[reprs] = sess.run([enc2], feed_dict = {tf_items: items})

In [20]:
reprs.shape

(10000, 150)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
def rmse(y_true, y_pred):
    """ Returns the root mean squared error """
    return np.sqrt(np.mean(np.square(y_true - y_pred)))

def mae(y_true, y_pred):
    """ Returns the mean absolute error """
    return np.mean(np.absolute(y_true - y_pred))

def evaluate(X_orig, X_held_out, sim, user_indices, item_indices):
    held_outs = X_orig[(user_indices, item_indices)]
    predictions = []
    for u, i in zip(user_indices, item_indices):
        user_arr = np.asarray(X_held_out[u,:].todense())[0]
        norm_const = np.sum((user_arr != 0) * (sim[i,:]))
        pred_rating = np.sum(user_arr * sim[i,:])/norm_const
        predictions.append(pred_rating)
    rmse_ = rmse(held_outs, np.array(predictions))
    mae_ = mae(held_outs, np.array(predictions))
    return np.array([rmse_, mae_])

sim = (cosine_similarity(reprs) + 1) / 2
evaluate(spr, X, sim, user_incides, item_indices)

array([0.90049046, 0.70004326], dtype=float32)